In [3]:
%load_ext autoreload
%autoreload 2
from model import DataLoader, FineTunedModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['c:\\Users\\user\\Desktop\\WN 22\\EECS 487\\eecs487-finalproject\\model', 'c:\\Python310\\python310.zip', 'c:\\Python310\\DLLs', 'c:\\Python310\\lib', 'c:\\Python310', '', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python310\\site-packages', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python310\\site-packages\\win32', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python310\\site-packages\\win32\\lib', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python310\\site-packages\\Pythonwin', 'c:\\Python310\\lib\\site-packages', '..', '..', '..']


In [5]:

data = DataLoader()



['c:\\Users\\user\\Desktop\\WN 22\\EECS 487\\eecs487-finalproject\\model', 'c:\\Python310\\python310.zip', 'c:\\Python310\\DLLs', 'c:\\Python310\\lib', 'c:\\Python310', '', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python310\\site-packages', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python310\\site-packages\\win32', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python310\\site-packages\\win32\\lib', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python310\\site-packages\\Pythonwin', 'c:\\Python310\\lib\\site-packages', '..', '..', '..', '..']
Found saved data
Successfully load pickle files
Successfully load unique_titles
Successfully load titles_id


In [6]:
# split the data into train, test, and validation 
from sklearn.model_selection import train_test_split

# split train as 50%, val as 25%, test as 25% 
def get_training_splits(data):
    X_train, X_test = train_test_split(
        data, test_size = 0.25, random_state=487, shuffle=True 
    )

    X_train, X_val = train_test_split(
        X_train, test_size = 0.33, random_state=487, shuffle=True
    )
    return X_train, X_val, X_test

X_train, X_val, X_test = get_training_splits(data.data)


In [13]:

from sklearn.neighbors import NearestNeighbors 
from collections import defaultdict

from model import SpecialDataLoader

special_dataloader = SpecialDataLoader()
special_X_train, special_X_val, special_X_test = get_training_splits(special_dataloader)

def special_performance(data_partition, k = 10):
    
    # knn = NearestNeighbors(n_neighbors=10, metric="cosine", n_jobs=-1, leaf_size=300)
    # knn.fit(cur_samples)
    mrr = 0
    acc = 0
    num_samples = 0
    # counts = defaultdict(int)
    for entry in data_partition:
        # print(entry)
        # counts[]
        # if the label is 0, we don't need to check 
        if entry["label"] == -1:
            continue
        num_samples += 1
        # first get the new embedding using our model
        # We use Q2 because Q2 is the duplicate 
        cur_embed = model.ff(entry["q1_embedding"])
        # we also need to get the real id 
        original_label = entry["q2_id"]
        
        nearest_neighbors = knn.kneighbors(cur_embed.detach().reshape(1, -1), k, return_distance=False).tolist()[0]
        
        # we need to start at the second index, because the first index is always gonan be itself 
        
        # acc
        if data.unique_embeddings_id[nearest_neighbors[1]] == original_label: 
            acc += 1
            
        # mrr
        for i, cur_neighbor in enumerate(nearest_neighbors[1:]):
            
            if data.unique_embeddings_id[cur_neighbor] == original_label:
                mrr += 1 / (1 + i)
                break
    
    mrr /= num_samples
    acc /= num_samples
    return acc, mrr

special_tacc, special_tmrr = special_performance(special_X_train)
print("Special | Training Accuracy", special_tacc, " | Training MRR:", special_tmrr)

{'q1_embedding': tensor([-0.6612, -0.4439, -0.9686,  0.7238,  0.7167, -0.2222,  0.7024,  0.2712,
        -0.8396, -0.9999, -0.4180,  0.9553,  0.9165,  0.8304,  0.6922, -0.7643,
        -0.1987, -0.6001,  0.2495,  0.2315,  0.6463,  1.0000, -0.2955,  0.3421,
         0.2795,  0.9798, -0.7503,  0.7382,  0.8752,  0.6916, -0.4653,  0.1631,
        -0.9689, -0.1673, -0.9767, -0.9771,  0.4652, -0.3426, -0.0740,  0.1112,
        -0.6656,  0.2647,  1.0000, -0.4610,  0.6262, -0.2689, -1.0000,  0.2466,
        -0.6560,  0.9377,  0.9460,  0.9306,  0.2956,  0.4798,  0.5133, -0.4116,
        -0.2058,  0.1331, -0.3158, -0.5013, -0.6846,  0.5949, -0.9109, -0.6035,
         0.9541,  0.9584, -0.2111, -0.3495, -0.1310, -0.0682,  0.7610,  0.1285,
        -0.4351, -0.8127,  0.7524,  0.2371, -0.7245,  1.0000, -0.5208, -0.9036,
         0.9678,  0.9117,  0.6605, -0.6597,  0.8889, -1.0000,  0.5640, -0.1380,
        -0.9508,  0.2594,  0.5715, -0.1929,  0.9742,  0.6347, -0.8308, -0.5404,
        -0.1857, -0.929

NameError: name 'knn' is not defined

In [5]:

from model import *
import torch
from sklearn.neighbors import NearestNeighbors 
from torch.optim import Adam
from torch.nn import CosineEmbeddingLoss
import numpy as np
from torch.nn.functional import normalize

"""
Possible solutions 
1) Use larger samples 
2) Batch it 
3) Look at other people's articles/publications 
4) BERT Large 

1) Inspect by plotting them 
2) Visually compare the data points 
"""

"""
Things to change
if underfitting 
1) lr 
2) number of hidden dimensions
3) number of hidden layers
4) ...

if overfitting 
1) weight decay
2) dropout
3) ...

"""
torch.manual_seed(487)
start_epoch = "1169"
lr = 3e-5
hd = 512
wd = 0.000001
dropout=0.05
model = FineTunedModel(hd, dropout=dropout)
# pathname = "save\\hd" + str(hd) + "_lr" + str(lr) + "_wd" + str(wd) + "_dropout" + str(dropout) + "_epoch"
pathname = "save\\hd" + str(hd) + "_lr" + str(lr) + "_wd" + str(wd) + "_epoch"

""" !!! LOAD MODEL HERE !!!! """
# filepath 
print("Loading model...")
model.load_state_dict(torch.load(pathname + start_epoch + ".pth"))
print("Model loaded successfully")
""" !!! LOAD MODEL HERE !!!! """

optim = Adam(model.parameters(), lr=lr, weight_decay=wd)
criterion = CosineEmbeddingLoss(reduction="mean") # cosine similarity 
# start the training process 
num_epoch = 5000

# TODO: do the actual data loading 

def performance(cur_samples, data_partition, k = 10):
    
    knn = NearestNeighbors(n_neighbors=10, metric="cosine", n_jobs=-1, leaf_size=300)
    knn.fit(cur_samples)
    mrr = 0
    acc = 0
    num_samples = 0
    for entry in data_partition:
        # if the label is 0, we don't need to check 
        if entry["label"] == -1:
            continue
        num_samples += 1
        # first get the new embedding using our model
        # We use Q2 because Q2 is the duplicate 
        # cur_embed = model.ff(entry["q1_embedding"])
        cur_embed = model.ff(entry["Q1_title"])
        # we also need to get the real id 
        # original_label = entry["q2_id"]
        original_label = entry["Q2_id"]
        
        nearest_neighbors = knn.kneighbors(cur_embed.detach().reshape(1, -1), k, return_distance=False).tolist()[0]
        
        # we need to start at the second index, because the first index is always gonan be itself 
        
        # acc
        if data.unique_embeddings_id[nearest_neighbors[1]] == original_label: 
            acc += 1
            
        # mrr
        for i, cur_neighbor in enumerate(nearest_neighbors[1:]):
            
            if data.unique_embeddings_id[cur_neighbor] == original_label:
                mrr += 1 / (1 + i)
                break
    
    mrr /= num_samples
    acc /= num_samples
    return acc, mrr

k = 10
best_loss = float("infinity")
best_acc = 0
best_mrr = 0 
best_epoch = 0
stats = {
    "best_loss": float("infinity"),
    "best_loss_ep": 0,
    "best_vacc": 0,
    "best_vacc_ep": 0,
    "best_tacc": 0,
    "best_tacc_ep": 0,
    "best_vmrr": 0,
    "best_vmrr_ep": 0,
    "best_tmrr": 0,
    "best_tmrr_ep": 0
}

def update_stats(key, value, ep):
    
    if value > stats[key]:
        stats[key+"_ep"] = ep 
        stats[key] = value 
        
# training loop 
for epoch in range(num_epoch):
    
    # before we start our loop, we need to retrain our nearest neighbors
    cur_samples =[] 
    
    with torch.no_grad():
        for unique_title in data.unique_embeddings:
            cur_samples.append(np.array(model.ff(unique_title)))
    # print("Cur sample first index", cur_samples[0])
    
    # now calcualte the mrr and accuracy
    vacc, vmrr = performance(cur_samples, X_val)
    tacc, tmrr = performance(cur_samples, X_train)
    print("Epoch:", epoch+1+int(start_epoch), " | Training Accuracy", tacc, " | Training MRR:", tmrr)
    print("Epoch:", epoch+1+int(start_epoch), " | Validation Accuracy", vacc, " | Validation MRR:", vmrr)

    update_stats("best_vacc", vacc, epoch+1+int(start_epoch))
    update_stats("best_tacc", tacc, epoch+1+int(start_epoch))
    update_stats("best_vmrr", vmrr, epoch+1+int(start_epoch))
    update_stats("best_tmrr", tmrr, epoch+1+int(start_epoch))
    
    # now start training process 
    total_loss = 0 
    for entry in X_train:   
        # q1 = entry["q1_embedding"]
        # q2 = entry["q2_embedding"]
        q1 = entry["Q1_title"]
        q2 = entry["Q2_title"]
        y = entry["label"]
        
        optim.zero_grad()   
        # this output a "new embedding" for both q1 and q2 
        out1, out2, sim = model(q1, q2)
        # print("Epoch:", epoch + 1, " | Similarity between q1 and q2 is:", sim, " | True label:", y)
        loss = criterion(out1, out2, torch.tensor(y, dtype=torch.float))
        loss.backward()
        
        if loss < stats["best_loss"]:
            stats["best_loss_ep"] = epoch+1+int(start_epoch) 
            stats[best_loss] = loss
            
        optim.step()
        total_loss += loss.item()
        
    try:
        torch.save(model.state_dict(), pathname + str(epoch + int(start_epoch) + 1) +".pth")
    except OSError as err:
        print("An error occured while saving model:")
        print(err)
        
    print("Epoch:", epoch+1+int(start_epoch), "with a loss of:", total_loss/len(data))
    
    # before we start our loop, we need to retrain our nearest neighbors
    cur_samples =[] 
    
    with torch.no_grad():
        for unique_embedding in data.unique_embeddings:
            cur_samples.append(np.array(model.ff(unique_embedding)))
    # print("Cur sample first index", cur_samples[0])
    # now calcualte the mrr and accuracy
    vacc, vmrr = performance(cur_samples, X_val)
    tacc, tmrr = performance(cur_samples, X_train)
    print("Epoch:", epoch+1+int(start_epoch), " | Training Accuracy", tacc, " | Training MRR:", tmrr)
    print("Epoch:", epoch+1+int(start_epoch), " | Validation Accuracy", vacc, " | Validation MRR:", vmrr)

    update_stats("best_vacc", vacc, epoch+1+int(start_epoch))
    update_stats("best_tacc", tacc, epoch+1+int(start_epoch))
    update_stats("best_vmrr", vmrr, epoch+1+int(start_epoch))
    update_stats("best_tmrr", tmrr, epoch+1+int(start_epoch))
    print("\n")
        

Loading model...
Model loaded successfully
Epoch: 1170  | Training Accuracy 0.005319148936170213  | Training MRR: 0.00775709219858156
Epoch: 1170  | Validation Accuracy 0.0  | Validation MRR: 0.001287001287001287
Epoch: 1170 with a loss of: 0.000474606734351255


Epoch: 1171  | Training Accuracy 0.005319148936170213  | Training MRR: 0.005984042553191489
Epoch: 1171  | Validation Accuracy 0.0  | Validation MRR: 0.0022522522522522522
Epoch: 1171 with a loss of: 2.499110996723175e-05


Epoch: 1172  | Training Accuracy 0.005319148936170213  | Training MRR: 0.007978723404255319
Epoch: 1172  | Validation Accuracy 0.0  | Validation MRR: 0.001287001287001287
Epoch: 1172 with a loss of: 0.00018470136390533298


Epoch: 1173  | Training Accuracy 0.010638297872340425  | Training MRR: 0.013076241134751771
Epoch: 1173  | Validation Accuracy 0.0  | Validation MRR: 0.0
Epoch: 1173 with a loss of: 0.000900263786315918


Epoch: 1174  | Training Accuracy 0.0  | Training MRR: 0.0007598784194528875
Epoch: 

In [16]:
# get the baseline metric 

from model import *
import torch
from sklearn.neighbors import NearestNeighbors 
from torch.optim import Adam
from torch.nn import CosineEmbeddingLoss
import numpy as np
from torch.nn.functional import normalize
torch.manual_seed(487)
def performance_no_finetune(cur_samples, data_partition, k = 10):
    
    knn = NearestNeighbors(n_neighbors=10, metric="cosine", n_jobs=-1, leaf_size=300)
    knn.fit(cur_samples)
    mrr = 0
    acc = 0
    num_samples = 0
    for entry in data_partition:
        # if the label is 0, we don't need to check 
        if entry["label"] == -1:
            continue
        num_samples += 1
        # first get the new embedding using our model
        # We use Q2 because Q2 is the duplicate 
        # cur_embed = model.ff(entry["Q1_title"])
        cur_embed = entry["Q1_title"]
        # we also need to get the real id 
        original_label = entry["Q2_id"]
        
        nearest_neighbors = knn.kneighbors(cur_embed.detach().reshape(1, -1), k, return_distance=False).tolist()[0]
        
        # we need to start at the second index, because the first index is always gonan be itself 
        
        # acc
        if data.unique_embeddings_id[nearest_neighbors[1]] == original_label: 
            acc += 1
            
        # mrr
        for i, cur_neighbor in enumerate(nearest_neighbors[1:]):
            
            if data.unique_embeddings_id[cur_neighbor] == original_label:
                mrr += 1 / (1 + i)
                break
    
    mrr /= num_samples
    acc /= num_samples
    return acc, mrr
cur_samples = []

for entry in data.data:
    cur_samples.append(np.array(entry["Q1_title"]))
    
vacc, vmrr = performance_no_finetune(cur_samples, X_val)
tacc, tmrr = performance_no_finetune(cur_samples, X_train)
print("Training Accuracy", tacc, " | Training MRR:", tmrr)
print("Validation Accuracy", vacc, " | Validation MRR:", vmrr)


Training Accuracy 0.0  | Training MRR: 0.005074299223235393
Validation Accuracy 0.0  | Validation MRR: 0.004129129129129129
